In [1]:
import pandas as pd
df=pd.read_csv('unigram_freq.csv')
Net_freq=df["count"].sum(skipna=True)
df.head()


,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698


<h1>The Model</h1>
<li>
 Identifying Misspelled Word</li>
<li>
 Finding strings which strings are n edits away<br>
 Create The function calculating edit distance for add, delete, swap and replace operations
 <br>
</li>
<li>
Filter Candidates: consider real words
</li>
<li>
Calculate Word Probablities:<br>
    candidate probablitiy=freq(candidate)/(sum of all freq)
    <br>
</li>



<h1>EDIT distance</h1>
<li>add, delete, replace: 1</li>


In [2]:
def editDistance(s1:str,s2:str):
    n1=len(s1);n2=len(s2)
    dp=[[0] * (n2 + 1) for _ in range(n1 + 1)]
    for i in range(1,n1+1):dp[i][0]=i
    for i in range(1,n2+1):dp[0][i]=i
    for i in range(1,n1+1):
        for j in range(1,n2+1):
            if(s1[i-1]==s2[j-1]):
                dp[i][j]=min(dp[i-1][j-1],dp[i-1][j]+1,dp[i][j-1]+1)
            else:
                dp[i][j]=min(dp[i-1][j-1]+1,dp[i-1][j]+1,dp[i][j-1]+1)

    return dp[n1][n2]


<h1>B K Tree</h1>

In [11]:
class BK_Tree_Node:
    def __init__(self,word:str,freq:int):
        self.children={}
        self.word=word
        self.freq=freq
        
class BK_Tree:
    def __init__(self,word:str,t:int,freq:int):
        self.t=t
        self.freq=freq
        self.word=word
        self.children={}
        
    def insert(self,word:str ,freq:int):
        curr=self
        ed=editDistance(word,curr.word)
        curr=self.children
        while ed in curr:
            edi=ed
            ed=editDistance(word,curr[edi].word)
            curr=curr[edi].children
        curr[ed]=BK_Tree_Node(word=word,freq=freq)
    def findWords(self,word:str,num=4):
        word=word.lower()
        edi=editDistance(word,self.word)
        possiblewords=[]
        t=self.t
        addresses=[[edi,self]]
        for i in addresses:
            [ed,curr]=i
            curr=curr.children
            low=0
            if(ed-t<0):low=0
            else:low=ed-t
            for j in range(low,ed+t+1):
                if j in curr:
                    edi=editDistance(curr[j].word,word)
                    if curr[j].word !=word :possiblewords.append([edi,1-curr[j].freq/Net_freq,curr[j].word])
                    addresses.append([edi,curr[j]])
        possiblewords.sort()
        for i in range(num):
             possiblewords[i]=possiblewords[i][2]

        return possiblewords[0:num]
    
                    

tree=BK_Tree("the",5,23135851162)
for i in df.iterrows():
    i[1]["word"]=str(i[1]["word"])
    i[1]["word"]=i[1]["word"].lower()
    tree.insert(word=i[1]["word"],freq=i[1]["count"])


<h1>Enter Your Word</h1>
<h4>Read the text below to know how to use it</h4>
<br>
<h3><b>Note : </b>Remember to run all the other code blocks above </h3>


In [13]:
tree.findWords("corporateon",4)
#Enter the string ^ here for which you want recommendations and enter number of recommendations you want as the next parameter  

['corporation', 'corporate', 'corporations', 'corporates']